In [ ]:
pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
# fetch dataset
bank_marketing = fetch_ucirepo(id=222)

# data (as pandas dataframes)
x = bank_marketing.data.features
y = bank_marketing.data.targets

df = pd.concat([x,y],axis=1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
file_path = '/content/drive/My Drive/bank_market.csv'
df.to_csv(file_path, index=False)

In [ ]:
df1 = pd.read_csv(file_path)

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e8e44d223b91b7466c095814591da9b8ae3b5d33731ae57dab00ee84a6abd82b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Data cleaning').getOrCreate()


In [ ]:
df=spark.read.csv(file_path,header=True,inferSchema=True)
df.show()
print(df.count())

+---+------------+--------+---------+-------+-------+-------+----+-------+-----------+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day_of_week|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+-----------+-----+--------+--------+-----+--------+--------+---+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|   NULL|          5|  may|     261|       1|   -1|       0|    NULL| no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|   NULL|          5|  may|     151|       1|   -1|       0|    NULL| no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|   NULL|          5|  may|      76|       1|   -1|       0|    NULL| no|
| 47| blue-collar| married|     NULL|     no|   1506|    yes|  no|   NULL|          5|  may|      92|       1|   -1|       0|    NULL| no|
| 33|        NULL|  single|

In [ ]:
select_column = df.select("age","job","marital","education","default","balance","housing","loan","duration")
select_column.show()

+---+------------+--------+---------+-------+-------+-------+----+--------+
|age|         job| marital|education|default|balance|housing|loan|duration|
+---+------------+--------+---------+-------+-------+-------+----+--------+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|     261|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|     151|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|      76|
| 47| blue-collar| married|     NULL|     no|   1506|    yes|  no|      92|
| 33|        NULL|  single|     NULL|     no|      1|     no|  no|     198|
| 35|  management| married| tertiary|     no|    231|    yes|  no|     139|
| 28|  management|  single| tertiary|     no|    447|    yes| yes|     217|
| 42|entrepreneur|divorced| tertiary|    yes|      2|    yes|  no|     380|
| 58|     retired| married|  primary|     no|    121|    yes|  no|      50|
| 43|  technician|  single|secondary|     no|    593|    yes|  no|      55|
| 41|      a

In [ ]:
from pyspark.sql.functions import col,mean,when,lit

df= df.fillna({'job':'unknown','marital':'unknown','education':'unknown','default':'unknown','housing':'unknown','loan':'unknown'})

mean_age = df.select(mean('age')).collect()[0][0]
mean_balance=df.select(mean('balance')).collect()[0][0]
mean_duration=df.select(mean('duration')).collect()[0][0]

df = df.withColumn('age',when(col('age').isNull(),mean_age).otherwise(col('age')))
df=df.withColumn('balance',when(col('balance').isNull(),mean_balance).otherwise(col('balance')))
df=df.withColumn('duration',when(col('duration').isNull(),mean_balance).otherwise(col('duration')))
q=df.select("age","job","marital","education","default","balance","housing","loan","duration")
q.show()

+----+------------+--------+---------+-------+-------+-------+----+--------+
| age|         job| marital|education|default|balance|housing|loan|duration|
+----+------------+--------+---------+-------+-------+-------+----+--------+
|58.0|  management| married| tertiary|     no| 2143.0|    yes|  no|   261.0|
|44.0|  technician|  single|secondary|     no|   29.0|    yes|  no|   151.0|
|33.0|entrepreneur| married|secondary|     no|    2.0|    yes| yes|    76.0|
|47.0| blue-collar| married|  unknown|     no| 1506.0|    yes|  no|    92.0|
|33.0|     unknown|  single|  unknown|     no|    1.0|     no|  no|   198.0|
|35.0|  management| married| tertiary|     no|  231.0|    yes|  no|   139.0|
|28.0|  management|  single| tertiary|     no|  447.0|    yes| yes|   217.0|
|42.0|entrepreneur|divorced| tertiary|    yes|    2.0|    yes|  no|   380.0|
|58.0|     retired| married|  primary|     no|  121.0|    yes|  no|    50.0|
|43.0|  technician|  single|secondary|     no|  593.0|    yes|  no|    55.0|

In [ ]:
from pyspark.sql.functions import sum,col

filtered_df=df.filter((col('job') != 'unknown') & (col ('age')>40))
q1= filtered_df.select("job","balance").groupby('job').agg(sum('balance'))
q1.show()

+-------------+------------+
|          job|sum(balance)|
+-------------+------------+
|   management|   8330997.0|
|      retired|   4472446.0|
|self-employed|   1385538.0|
|      student|     15723.0|
|  blue-collar|   5350915.0|
| entrepreneur|   1457319.0|
|       admin.|   2959149.0|
|   technician|   4040625.0|
|     services|   1938982.0|
|    housemaid|   1189372.0|
|   unemployed|   1045435.0|
+-------------+------------+



In [ ]:
#Encoding Categorical Variables:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

categorical_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan']
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid='keep') for col in categorical_cols]
encoder = OneHotEncoder(inputCols=[col+"_index" for col in categorical_cols], outputCols=[col+"_encoded" for col in categorical_cols])

pipeline = Pipeline(stages=indexers + [encoder])
df_encoded = pipeline.fit(df).transform(df)

# Select and display the specified columns
columns_to_show = categorical_cols + [col+"_index" for col in categorical_cols] + [col+"_encoded" for col in categorical_cols]
df_encoded.select(columns_to_show).show(truncate=False)


+------------+--------+---------+-------+-------+----+---------+-------------+---------------+-------------+-------------+----------+---------------+---------------+-----------------+---------------+---------------+-------------+
|job         |marital |education|default|housing|loan|job_index|marital_index|education_index|default_index|housing_index|loan_index|job_encoded    |marital_encoded|education_encoded|default_encoded|housing_encoded|loan_encoded |
+------------+--------+---------+-------+-------+----+---------+-------------+---------------+-------------+-------------+----------+---------------+---------------+-----------------+---------------+---------------+-------------+
|management  |married |tertiary |no     |yes    |no  |1.0      |0.0          |1.0            |0.0          |0.0          |0.0       |(12,[1],[1.0]) |(3,[0],[1.0])  |(4,[1],[1.0])    |(2,[0],[1.0])  |(2,[0],[1.0])  |(2,[0],[1.0])|
|technician  |single  |secondary|no     |yes    |no  |2.0      |1.0          |0.

In [ ]:
#Scaling Numerical Variables:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import udf

# Assemble numerical columns into a vector for scaling
assembler = VectorAssembler(inputCols=['age', 'balance', 'duration'], outputCol='features')
df_vector = assembler.transform(df_encoded)

# Apply StandardScaler
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')
scaler_model = scaler.fit(df_vector)
df_scaled = scaler_model.transform(df_vector)

df_scaled.select('age', 'balance', 'duration', 'features', 'scaled_features').show(truncate=False)



+----+-------+--------+-------------------+-------------------------------------------------------------+
|age |balance|duration|features           |scaled_features                                              |
+----+-------+--------+-------------------+-------------------------------------------------------------+
|58.0|2143.0 |261.0   |[58.0,2143.0,261.0]|[5.4620302984652165,0.7038308100643741,1.0134827679553768]   |
|44.0|29.0   |151.0   |[44.0,29.0,151.0]  |[4.143609191939129,0.009524541993404969,0.5863444366331874]  |
|33.0|2.0    |76.0    |[33.0,2.0,76.0]    |[3.1077068939543473,6.568649650624117E-4,0.29511375618624003]|
|47.0|1506.0 |92.0    |[47.0,1506.0,92.0] |[4.426128000480434,0.494619318691996,0.3572429680149221]     |
|33.0|1.0    |198.0   |[33.0,1.0,198.0]   |[3.1077068939543473,3.2843248253120583E-4,0.7688489963799411]|
|35.0|231.0  |139.0   |[35.0,231.0,139.0] |[3.2960527663152166,0.07586790346470855,0.5397475277616758]  |
|28.0|447.0  |217.0   |[28.0,447.0,217.0] |[2.

In [ ]:
# Creating Derived Features:
from pyspark.sql.functions import when

# Example: Binning age into age groups
df = df.withColumn('age_group', when(col('age') <= 25, '18-25')
                                .when((col('age') > 25) & (col('age') <= 35), '26-35')
                                .when((col('age') > 35) & (col('age') <= 45), '36-45')
                                .otherwise('46+'))

df.select('age', 'age_group').show()

+----+---------+
| age|age_group|
+----+---------+
|58.0|      46+|
|44.0|    36-45|
|33.0|    26-35|
|47.0|      46+|
|33.0|    26-35|
|35.0|    26-35|
|28.0|    26-35|
|42.0|    36-45|
|58.0|      46+|
|43.0|    36-45|
|41.0|    36-45|
|29.0|    26-35|
|53.0|      46+|
|58.0|      46+|
|57.0|      46+|
|51.0|      46+|
|45.0|    36-45|
|57.0|      46+|
|60.0|      46+|
|33.0|    26-35|
+----+---------+
only showing top 20 rows

